In [291]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [292]:
#Import libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PolynomialFeatures,RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.linear_model import LassoCV,RidgeCV,ElasticNetCV
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.model_selection import GridSearchCV,KFold,RandomizedSearchCV, StratifiedKFold,cross_val_score
from sklearn.metrics import r2_score
sns.set_context("paper", font_scale = 1, rc={"grid.linewidth": 3})
pd.set_option('display.max_rows', 100, 'display.max_columns', 400)
from scipy.stats import skew,boxcox_normmax
from scipy.special import boxcox1p
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRFRegressor,XGBRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor

In [293]:
df_train = pd.read_csv("/content/drive/MyDrive/Datasets/Finance/train_Data.csv",parse_dates=['DisbursalDate','MaturityDAte','AuthDate'])

df_test = pd.read_csv("/content/drive/MyDrive/Datasets/Finance/test_Data.csv",parse_dates=['DisbursalDate','MaturityDAte','AuthDate'])


types = {"ID":"int32","WRITE-OFF-AMT":"float32","TENURE":"float16"}
df_train_bureau = pd.read_csv("/content/drive/MyDrive/Datasets/Finance/train_bureau.csv",dtype=types,parse_dates=['DATE-REPORTED','DISBURSED-DT','CLOSE-DT','LAST-PAYMENT-DATE'])

types = {"ID":"int32","WRITE-OFF-AMT":"float32","TENURE":"float16"}
df_test_bureau = pd.read_csv("/content/drive/MyDrive/Datasets/Finance/test_bureau.csv",dtype=types,parse_dates=['DATE-REPORTED','DISBURSED-DT','CLOSE-DT','LAST-PAYMENT-DATE'])

In [294]:
Submission = df_test[['ID']]

#Combining the train and test datasets for better EDA
df_train['Source'] = "Train"
df_test['Source'] = "Test"

df = pd.concat([df_train, df_test],axis=0)
df.shape
df = df.drop_duplicates()

In [295]:
df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month,Source
0,1,Monthly,Arrear,Closed,PDC_E,1,NaN,48,450000,275000.0,275000.0,24000.0,2012-02-10,2016-01-15,2012-02-10,4022465,1568.0,21946,61.11,M,49.0,35833.33,RAISEN,MADHYA PRADESH,464993.0,> 48 Months,Train
1,2,Monthly,Advance,Closed,PDC,333,BHOPAL,47,485000,350000.0,350000.0,10500.0,2012-03-31,2016-02-15,2012-03-31,4681175,1062.0,34802,70.00,M,23.0,666.67,SEHORE,MADHYA PRADESH,466001.0,No Top-up Service,Train
2,3,Quatrly,Arrear,Active,Direct Debit,1,NaN,68,690000,519728.0,519728.0,38300.0,2017-06-17,2023-02-10,2017-06-17,25328146,1060.0,127335,69.77,M,39.0,45257.00,BHOPAL,MADHYA PRADESH,462030.0,12-18 Months,Train
3,7,Monthly,Advance,Closed,Billed,125,GUNA,48,480000,400000.0,400000.0,11600.0,2013-11-29,2017-11-10,2013-11-29,13021591,1060.0,25094,80.92,M,24.0,20833.33,ASHOK NAGAR,MADHYA PRADESH,473335.0,> 48 Months,Train
4,8,Monthly,Arrear,Closed,Billed,152,BILASPUR,44,619265,440000.0,440000.0,15000.0,2011-12-08,2015-07-05,2011-12-08,3291320,1046.0,21853,71.05,M,56.0,27313.67,BILASPUR,CHATTISGARH,495442.0,36-48 Months,Train


**EDA - Bureau Data**

1. Data-Cleaning
2. Feature Engineering and Feature Selection
(Starting with null value imputation)
3. Encoding and further

In [296]:
#Dropping duplicates, if any
df_train_bureau = df_train_bureau.drop_duplicates()
df_test_bureau = df_test_bureau.drop_duplicates()

In [297]:
df_train_bureau.shape, df_test_bureau.shape

((556980, 25), (63331, 25))

In [298]:
df_train_bureau['Source'] = "Train"
df_test_bureau['Source'] = "Test"
df_bureau_data = pd.concat([df_train_bureau, df_test_bureau],axis=0)
df_bureau_data.shape

(620311, 26)

In [299]:
df_bureau_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 620311 entries, 0 to 64018
Data columns (total 26 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   ID                         620311 non-null  int32         
 1   SELF-INDICATOR             620311 non-null  bool          
 2   MATCH-TYPE                 620311 non-null  object        
 3   ACCT-TYPE                  620311 non-null  object        
 4   CONTRIBUTOR-TYPE           620311 non-null  object        
 5   DATE-REPORTED              616634 non-null  datetime64[ns]
 6   OWNERSHIP-IND              620311 non-null  object        
 7   ACCOUNT-STATUS             620311 non-null  object        
 8   DISBURSED-DT               588384 non-null  datetime64[ns]
 9   CLOSE-DT                   340799 non-null  object        
 10  LAST-PAYMENT-DATE          269683 non-null  datetime64[ns]
 11  CREDIT-LIMIT/SANC AMT      16688 non-null   object   

  1. We can see there are 18 variables that are of object dtype and few are of datetime dtype followed by 1 each in float and int category. 
  2. We can also see that there are many variables with missing values. If the percentage of missing values is more than 50%, we will go ahead and delete the variable. 
  3. We also have to check the object variables, about the cardinality. If there are many values, we might have to combine a few of them. 

In [300]:
df_bureau_data.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,Source
0,1,False,PRIMARY,Overdraft,NAB,2018-04-30,Individual,Delinquent,2015-10-05,NaN,2018-02-27,NaN,"37,352",NaN,"37,873",NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN,Train
1,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31,Individual,Active,2018-03-19,NaN,2019-12-19,NaN,"44,000","1,405/Monthly","20,797",F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",36.0,Train
2,1,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2019-08-30,NaN,NaT,NaN,"1,45,000",NaN,"1,16,087",NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",NaN,Train
3,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30,Individual,Closed,2013-09-27,2017-09-21 00:00:00,NaT,NaN,"3,00,000",NaN,0,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,Train
4,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29,Individual,Closed,2012-02-10,2016-02-01 00:00:00,NaT,NaN,"2,75,000",NaN,0,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,Train


In [301]:
(df_bureau_data.isna().sum()/len(df_bureau_data))*100

ID                            0.000000
SELF-INDICATOR                0.000000
MATCH-TYPE                    0.000000
ACCT-TYPE                     0.000000
CONTRIBUTOR-TYPE              0.000000
DATE-REPORTED                 0.592767
OWNERSHIP-IND                 0.000000
ACCOUNT-STATUS                0.000000
DISBURSED-DT                  5.146934
CLOSE-DT                     45.059978
LAST-PAYMENT-DATE            56.524550
CREDIT-LIMIT/SANC AMT        97.309737
DISBURSED-AMT/HIGH CREDIT     0.006126
INSTALLMENT-AMT              75.177613
CURRENT-BAL                   0.042559
INSTALLMENT-FREQUENCY        76.010098
OVERDUE-AMT                  21.128756
WRITE-OFF-AMT                 3.485832
ASSET_CLASS                  53.883294
REPORTED DATE - HIST          3.485832
DPD - HIST                    3.559666
CUR BAL - HIST                3.485832
AMT OVERDUE - HIST            3.485832
AMT PAID - HIST               3.691697
TENURE                       65.981097
Source                   

In [302]:
df_bureau_data.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,Source
0,1,False,PRIMARY,Overdraft,NAB,2018-04-30,Individual,Delinquent,2015-10-05,NaN,2018-02-27,NaN,"37,352",NaN,"37,873",NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN,Train
1,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31,Individual,Active,2018-03-19,NaN,2019-12-19,NaN,"44,000","1,405/Monthly","20,797",F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",36.0,Train
2,1,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2019-08-30,NaN,NaT,NaN,"1,45,000",NaN,"1,16,087",NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",NaN,Train
3,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30,Individual,Closed,2013-09-27,2017-09-21 00:00:00,NaT,NaN,"3,00,000",NaN,0,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,Train
4,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29,Individual,Closed,2012-02-10,2016-02-01 00:00:00,NaT,NaN,"2,75,000",NaN,0,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,Train


In [303]:
for col in df_bureau_data.columns:
    if df_bureau_data[col].dtype == "object":
        print ("Value Count of {} column are:\n{}".format(col, df_bureau_data[col].value_counts(dropna=False)))

Value Count of MATCH-TYPE column are:
PRIMARY      620086
SECONDARY       225
Name: MATCH-TYPE, dtype: int64
Value Count of ACCT-TYPE column are:
Tractor Loan                                                           207869
Gold Loan                                                               98370
Business Loan Priority Sector  Agriculture                              88083
Kisan Credit Card                                                       37588
Auto Loan (Personal)                                                    33432
Other                                                                   29602
Commercial Vehicle Loan                                                 20484
Two-Wheeler Loan                                                        17641
Credit Card                                                             14081
Consumer Loan                                                           13675
Overdraft                                                               10

In [304]:
#We will drop the MATCH-TYPE column as it has one value which is dominating
df_bureau_data.drop("MATCH-TYPE",axis=1,inplace=True)

#We will combine the values of certain columns which are few in number
df_bureau_data['ACCT-TYPE'].replace({"Secured Credit Card":"Others",
                                  "Business Non-Funded Credit Facility General":"Others",
                                  "Prime Minister Jaan Dhan Yojana - Overdraft":"Others",
                                  "Pradhan Mantri Awas Yojana - CLSS":"Others",
                                  "SHG Individual":"Others",
                                  "JLG Group":"Others",
                                  "Microfinance Personal Loan":"Others",
                                  "Fleet Card":"Others",
                                  "Microfinance Housing Loan":"Others",
                                  "Commercial Equipment Loan":"Others",
                                  "Corporate Credit Card":"Others",
                                  "Loan on Credit Card":"Others",
                                  "Business Non-Funded Credit Facility-Priority Sector-Others":"Others",
                                  "Leasing":"Others",
                                  "Telco Landline":"Others",
                                  "SHG Group":"Others",
                                  "Staff Loan":"Others"},inplace=True)
df_bureau_data['CONTRIBUTOR-TYPE'].replace({"SFB":"Others",
                                         "ARC":"Others",
                                         "OFI":"Others"},inplace=True)
df_bureau_data['ACCOUNT-STATUS'].replace({"SUIT FILED (WILFUL DEFAULT)":"Others",
                                       "WILFUL DEFAULT":"Others",
                                       "Sold/Purchased":"Others",
                                       "Cancelled":"Others"},inplace=True)

# We will drop the ASSET_CLASS variable as it has majority of missing values.
df_bureau_data.drop("ASSET_CLASS",axis=1,inplace=True)

In [305]:
df_bureau_data.isna().sum()/len(bureau_data)

ID                           0.000000
SELF-INDICATOR               0.000000
ACCT-TYPE                    0.000000
CONTRIBUTOR-TYPE             0.000000
DATE-REPORTED                0.005928
OWNERSHIP-IND                0.000000
ACCOUNT-STATUS               0.000000
DISBURSED-DT                 0.051469
CLOSE-DT                     0.450600
LAST-PAYMENT-DATE            0.565245
CREDIT-LIMIT/SANC AMT        0.973097
DISBURSED-AMT/HIGH CREDIT    0.000061
INSTALLMENT-AMT              0.751776
CURRENT-BAL                  0.000426
INSTALLMENT-FREQUENCY        0.760101
OVERDUE-AMT                  0.211288
WRITE-OFF-AMT                0.034858
REPORTED DATE - HIST         0.034858
DPD - HIST                   0.035597
CUR BAL - HIST               0.034858
AMT OVERDUE - HIST           0.034858
AMT PAID - HIST              0.036917
TENURE                       0.659811
Source                       0.000000
dtype: float64

In [306]:
df_bureau_data.drop(['TENURE','OVERDUE-AMT','INSTALLMENT-FREQUENCY','CREDIT-LIMIT/SANC AMT','LAST-PAYMENT-DATE','CLOSE-DT'],axis=1,inplace=True)

In [307]:
#We will delete the Date Variables as of now
df_bureau_data.drop(['DATE-REPORTED','DISBURSED-DT','REPORTED DATE - HIST'],axis=1,inplace=True)

In [308]:
df_bureau_data.head()

,ID,SELF-INDICATOR,ACCT-TYPE,CONTRIBUTOR-TYPE,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,WRITE-OFF-AMT,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,Source
0,1,False,Overdraft,NAB,Individual,Delinquent,"37,352",NaN,"37,873",0.0,030000,"37873,12820,","37873,,",",,",Train
1,1,False,Auto Loan (Personal),NAB,Individual,Active,"44,000","1,405/Monthly","20,797",0.0,0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",Train
2,1,True,Tractor Loan,NBF,Individual,Active,"1,45,000",NaN,"1,16,087",0.0,000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",Train
3,1,True,Auto Loan (Personal),NBF,Individual,Closed,"3,00,000",NaN,0,0.0,000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",Train
4,1,True,Tractor Loan,NBF,Individual,Closed,"2,75,000",NaN,0,0.0,0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",Train


In [309]:
df_bureau_data.drop(df_bureau_data[df_bureau_data['WRITE-OFF-AMT']<0].index,axis=0,inplace=True)

In [310]:
df_bureau_data[df_bureau_data['WRITE-OFF-AMT']<0]

,ID,SELF-INDICATOR,ACCT-TYPE,CONTRIBUTOR-TYPE,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,WRITE-OFF-AMT,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,Source


In [311]:
df_bureau_data.head(n=10)

,ID,SELF-INDICATOR,ACCT-TYPE,CONTRIBUTOR-TYPE,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,WRITE-OFF-AMT,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,Source
0,1,False,Overdraft,NAB,Individual,Delinquent,"37,352",NaN,"37,873",0.0,030000,"37873,12820,","37873,,",",,",Train
1,1,False,Auto Loan (Personal),NAB,Individual,Active,"44,000","1,405/Monthly","20,797",0.0,0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",Train
2,1,True,Tractor Loan,NBF,Individual,Active,"1,45,000",NaN,"1,16,087",0.0,000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",Train
3,1,True,Auto Loan (Personal),NBF,Individual,Closed,"3,00,000",NaN,0,0.0,000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",Train
4,1,True,Tractor Loan,NBF,Individual,Closed,"2,75,000",NaN,0,0.0,0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",Train
5,1,False,Credit Card,NAB,Individual,Closed,0,NaN,0,0.0,000000000,"24650,17300,0,",",,,",",,,,",Train
6,1,False,Auto Loan (Personal),NAB,Individual,Active,"5,00,000","7,934/Monthly","4,43,769",0.0,000000000000000000000000000000000000000000,"443769,448365,453134,457687,462320,466802,4712...",",,,,,,,,,,,,,,",",,,,,,,,,,,,,,",Train
7,1,True,Auto Loan (Personal),NBF,Individual,Closed,"5,00,000",NaN,0,0.0,000DDD0000000000000870540270270000320000000000...,"0,,0,0,0,0,0,0,0,13060,13060,25982,38767,51417...","0,,0,0,0,0,3064,3064,3064,2972,0,2802,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",Train
8,1,True,Auto Loan (Personal),NBF,Individual,Closed,"4,00,000",NaN,0,0.0,000DDD0220210220190220220000270000000000000000...,"0,,0,10487,20862,31126,41281,51328,51328,71101...","0,,1722,1722,1722,1722,1722,1722,0,1625,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",Train
9,2,False,Overdraft,PRB,Individual,Closed,"17,00,000",NaN,0,0.0,000DDDDDD000DDDDDD000DDDDDD000DDDDDD000,"0,,,1699997,,,154997,,,-3,,,149997,","0,,,0,,,0,,,0,,,0",",,,,,,,,,,,,,,",Train


In [312]:
#Some of the numerical variables are coded as of object type because of the comma in the numbers, we convert them to the right data type.
df_bureau_data['DISBURSED-AMT/HIGH CREDIT'] = df_bureau_data['DISBURSED-AMT/HIGH CREDIT'].str.replace(",","")
df_bureau_data['DISBURSED-AMT/HIGH CREDIT'] = pd.to_numeric(df_bureau_data['DISBURSED-AMT/HIGH CREDIT'])

df_bureau_data['CURRENT-BAL'] = df_bureau_data['CURRENT-BAL'].str.replace(",","")
df_bureau_data['CURRENT-BAL'] = pd.to_numeric(df_bureau_data['CURRENT-BAL'])

In [313]:
df_bureau_data['DISBURSED-AMT/HIGH CREDIT'].fillna(df_bureau_data['DISBURSED-AMT/HIGH CREDIT'].mean(),inplace=True)
df_bureau_data['CURRENT-BAL'].fillna(df_bureau_data['CURRENT-BAL'].mean(),inplace=True)
df_bureau_data['WRITE-OFF-AMT'].fillna(df_bureau_data['WRITE-OFF-AMT'].mean(),inplace=True)

In [314]:
df_bureau_data.isna().sum()

ID                                0
SELF-INDICATOR                    0
ACCT-TYPE                         0
CONTRIBUTOR-TYPE                  0
OWNERSHIP-IND                     0
ACCOUNT-STATUS                    0
DISBURSED-AMT/HIGH CREDIT         0
INSTALLMENT-AMT              466333
CURRENT-BAL                       0
WRITE-OFF-AMT                     0
DPD - HIST                    22081
CUR BAL - HIST                21623
AMT OVERDUE - HIST            21623
AMT PAID - HIST               22900
Source                            0
dtype: int64

In [315]:
df_train_bureau = df_bureau_data[df_bureau_data['Source']=="Train"]
df_test_bureau = df_bureau_data[df_bureau_data['Source']=="Test"]
df_train_bureau.drop("Source",axis=1,inplace=True)
df_test_bureau.drop("Source",axis=1,inplace=True)

In [316]:
# Data-Cleaning
def ddp_hist_func(value):
    history =  [value[i:i+3] for i in range(0,len(value),3)]
    count_no_delays = len([c for c in history if c=="000"])
    count_no_payment_history_this_month = len([c for c in history if c=="DDD"])
    count_no_payment_history_prior = len([c for c in history if c=="XXX"])
    count_payments_past_due_date = len(history)-count_no_delays-count_no_payment_history_this_month-count_no_payment_history_prior
    return count_no_delays,count_payments_past_due_date,count_no_payment_history_this_month,count_no_payment_history_prior

df_train_bureau['DPD - HIST'].fillna(",",inplace=True)
df_train_bureau['DPD_History_Months'] = df_train_bureau['DPD - HIST'].apply(ddp_hist_func)

ddp_hist =pd.DataFrame(df_train_bureau['DPD_History_Months'].tolist())
ddp_hist.columns = ['Count_No_Delays_In_Payment','Count_Payment_Past_Due_Date','History_Not_Available_Curr_Month','History_Not_Available_Prior_Month']
df_train_bureau = pd.concat([df_train_bureau,ddp_hist],axis=1)
df_train_bureau.drop('DPD_History_Months',axis=1,inplace=True)


def cur_bal_func(val):
    history = val.split(",")[:]
    history_lst = [int(item) if item!="" else 0 for item in history]
    
    max_cur_bal = max(np.array(history_lst))
    min_cur_bal = min(np.array(history_lst))
    sum_cur_bal = sum(np.array(history_lst))
    avg_cur_bal = np.mean(np.array(history_lst))
    
    sum_successive_diff = np.sum(np.diff(np.array(history_lst)))
    avg_successive_diff = sum_successive_diff/len(np.diff(np.array(history_lst)))
    
    return min_cur_bal,max_cur_bal,sum_cur_bal,avg_cur_bal,sum_successive_diff,avg_successive_diff

df_train_bureau['CUR BAL - HIST'].fillna(",",inplace=True)
df_train_bureau['CUR_Bal_History_Months'] = df_train_bureau['CUR BAL - HIST'].apply(cur_bal_func)

cur_bal_hist = pd.DataFrame(df_train_bureau['CUR_Bal_History_Months'].tolist())
cur_bal_hist.columns = ['Min_Cur_Bal','Max_Cur_Bal','Sum_Cur_Bal','Avg_Cur_Bal','Sum_Successive_Diff_CurBal','Avg_Successive_Diff_CurBal']
df_train_bureau = pd.concat([df_train_bureau, cur_bal_hist],axis=1)
df_train_bureau.drop('CUR_Bal_History_Months',axis=1,inplace=True)


def amt_overdue_func(val):
    history = val.split(",")
    history_lst = [int(item) if item!="" else 0 for item in history]
    
    max_overdue = max(np.array(history_lst))
    min_overdue = min(np.array(history_lst))
    sum_overdue = sum(np.array(history_lst))
    avg_overdue = np.mean(np.array(history_lst))
    
    sum_successive_diff = np.sum(np.diff(np.array(history_lst)))
    avg_successive_diff = sum_successive_diff/len(np.diff(np.array(history_lst)))
        
    return min_overdue,max_overdue,sum_overdue,avg_overdue,sum_successive_diff,avg_successive_diff

df_train_bureau['AMT OVERDUE - HIST'].fillna(",",inplace=True)
df_train_bureau['AMT_Overdue_History_Months'] = df_train_bureau['AMT OVERDUE - HIST'].apply(amt_overdue_func)

amt_overdue_hist =pd.DataFrame(df_train_bureau['AMT_Overdue_History_Months'].tolist())
amt_overdue_hist.columns = ['Min_Overdue','Max_Overdue','Sum_Overdue','Avg_Overdue','Sum_Successive_Diff_Overdue','Avg_Successive_Diff_Overdue']
df_train_bureau = pd.concat([df_train_bureau, amt_overdue_hist],axis=1)
df_train_bureau.drop('AMT_Overdue_History_Months',axis=1,inplace=True)


def amt_paid_func(val):
    history = val.split(",")
    history_lst = [int(float(item)) if item!="" else 0 for item in history]
    
    max_amount_paid = max(np.array(history_lst))
    min_amount_paid = min(np.array(history_lst))
    sum_amount_paid = sum(np.array(history_lst))
    avg_amount_paid = np.mean(np.array(history_lst))
    
    return min_amount_paid,max_amount_paid,sum_amount_paid,avg_amount_paid

df_train_bureau['AMT PAID - HIST'].fillna(",",inplace=True)
df_train_bureau['AMT_Paid_History_Months'] = df_train_bureau['AMT PAID - HIST'].apply(amt_paid_func)

#This is the way how you transform and rejoin a column to the dataframe in pandas
amt_paid_hist = pd.DataFrame(df_train_bureau['AMT_Paid_History_Months'].tolist())
amt_paid_hist.columns = ['Min_Amount_Paid','Max_Amount_Paid','Sum_Amount_Paid','Avg_Amount_Paid']
df_train_bureau = pd.concat([df_train_bureau,amt_paid_hist],axis=1)
df_train_bureau.drop('AMT_Paid_History_Months',axis=1,inplace=True)


def installment_amt_func(val):
    amount = val.split("/")[0]
    duration = val.split("/")[-1]
    return amount,duration

df_train_bureau['INSTALLMENT-AMT'].fillna("",inplace=True)
df_train_bureau['INSTALLMENT_HIST'] = df_train_bureau['INSTALLMENT-AMT'].apply(installment_amt_func)

installment_amt_hist = pd.DataFrame(df_train_bureau['INSTALLMENT_HIST'].tolist())
installment_amt_hist.columns = ['Installment_Amount','Installment_Frequency']
df_train_bureau = pd.concat([df_train_bureau, installment_amt_hist],axis=1)
df_train_bureau.drop(['INSTALLMENT_HIST','Installment_Frequency'],axis=1,inplace=True)

del installment_amt_hist,ddp_hist,amt_paid_hist,amt_overdue_hist,cur_bal_hist #Dropping unnecesdary columns

In [317]:
df_train_bureau.drop(['AMT PAID - HIST','AMT OVERDUE - HIST','CUR BAL - HIST','DPD - HIST'],axis=1,inplace=True)

In [318]:
def ddp_hist_func(value):
    history =  [value[i:i+3] for i in range(0,len(value),3)]
    count_no_delays = len([c for c in history if c=="000"])
    count_no_payment_history_this_month = len([c for c in history if c=="DDD"])
    count_no_payment_history_prior = len([c for c in history if c=="XXX"])
    count_payments_past_due_date = len(history)-count_no_delays-count_no_payment_history_this_month-count_no_payment_history_prior
    return count_no_delays,count_payments_past_due_date,count_no_payment_history_this_month,count_no_payment_history_prior

df_test_bureau['DPD - HIST'].fillna(",",inplace=True)
df_test_bureau['DPD_History_Months'] = df_test_bureau['DPD - HIST'].apply(ddp_hist_func)

ddp_hist = pd.DataFrame(df_test_bureau['DPD_History_Months'].tolist())
ddp_hist.columns = ['Count_No_Delays_In_Payment','Count_Payment_Past_Due_Date','History_Not_Available_Curr_Month','History_Not_Available_Prior_Month']
df_test_bureau = pd.concat([df_test_bureau,ddp_hist],axis=1)
df_test_bureau.drop('DPD_History_Months',axis=1,inplace=True)


def cur_bal_func(val):
    history = val.split(",")[:-1]
    history_lst = [int(item) if item!="" else 0 for item in history]
    
    max_cur_bal = max(np.array(history_lst))
    min_cur_bal = min(np.array(history_lst))
    sum_cur_bal = sum(np.array(history_lst))
    avg_cur_bal = np.mean(np.array(history_lst))
    
    sum_successive_diff = np.sum(np.diff(np.array(history_lst)))
    avg_successive_diff = sum_successive_diff/len(np.diff(np.array(history_lst)))
    
    return min_cur_bal,max_cur_bal,sum_cur_bal,avg_cur_bal,sum_successive_diff,avg_successive_diff

df_test_bureau['CUR BAL - HIST'].fillna(",",inplace=True)
df_test_bureau['CUR_Bal_History_Months'] = df_test_bureau['CUR BAL - HIST'].apply(cur_bal_func)

cur_bal_hist = pd.DataFrame(df_test_bureau['CUR_Bal_History_Months'].tolist())
cur_bal_hist.columns = ['Min_Cur_Bal','Max_Cur_Bal','Sum_Cur_Bal','Avg_Cur_Bal','Sum_Successive_Diff_CurBal','Avg_Successive_Diff_CurBal']
df_test_bureau = pd.concat([df_test_bureau, cur_bal_hist],axis=1)
df_test_bureau.drop('CUR_Bal_History_Months',axis=1,inplace=True)


def amt_overdue_func(val):
    history = val.split(",")
    history_lst = [int(item) if item!="" else 0 for item in history]
    
    max_overdue = max(np.array(history_lst))
    min_overdue = min(np.array(history_lst))
    sum_overdue = sum(np.array(history_lst))
    avg_overdue = np.mean(np.array(history_lst))
    
    sum_successive_diff = np.sum(np.diff(np.array(history_lst)))
    avg_successive_diff = sum_successive_diff/len(np.diff(np.array(history_lst)))
        
    return min_overdue,max_overdue,sum_overdue,avg_overdue,sum_successive_diff,avg_successive_diff

df_test_bureau['AMT OVERDUE - HIST'].fillna(",",inplace=True)
df_test_bureau['AMT_Overdue_History_Months'] = df_test_bureau['AMT OVERDUE - HIST'].apply(amt_overdue_func)

amt_overdue_hist =pd.DataFrame(df_test_bureau['AMT_Overdue_History_Months'].tolist())
amt_overdue_hist.columns = ['Min_Overdue','Max_Overdue','Sum_Overdue','Avg_Overdue','Sum_Successive_Diff_Overdue','Avg_Successive_Diff_Overdue']
df_test_bureau = pd.concat([df_test_bureau, amt_overdue_hist],axis=1)
df_test_bureau.drop('AMT_Overdue_History_Months',axis=1,inplace=True)


def amt_paid_func(val):
    history = val.split(",")
    history_lst = [int(float(item)) if item!="" else 0 for item in history]
    
    max_amount_paid = max(np.array(history_lst))
    min_amount_paid = min(np.array(history_lst))
    sum_amount_paid = sum(np.array(history_lst))
    avg_amount_paid = np.mean(np.array(history_lst))
    
    return min_amount_paid,max_amount_paid,sum_amount_paid,avg_amount_paid

df_test_bureau['AMT PAID - HIST'].fillna(",",inplace=True)
df_test_bureau['AMT_Paid_History_Months'] = df_test_bureau['AMT PAID - HIST'].apply(amt_paid_func)

amt_paid_hist = pd.DataFrame(df_test_bureau['AMT_Paid_History_Months'].tolist())
amt_paid_hist.columns = ['Min_Amount_Paid','Max_Amount_Paid','Sum_Amount_Paid','Avg_Amount_Paid']
df_test_bureau = pd.concat([df_test_bureau, amt_paid_hist],axis=1)
df_test_bureau.drop('AMT_Paid_History_Months',axis=1,inplace=True)


def installment_amt_func(val):
    amount = val.split("/")[0]
    duration = val.split("/")[-1]
    return amount,duration

df_test_bureau['INSTALLMENT-AMT'].fillna("",inplace=True)
df_test_bureau['INSTALLMENT_HIST'] = df_test_bureau['INSTALLMENT-AMT'].apply(installment_amt_func)

installment_amt_hist = pd.DataFrame(df_test_bureau['INSTALLMENT_HIST'].tolist())
installment_amt_hist.columns = ['Installment_Amount','Installment_Frequency']
df_test_bureau = pd.concat([df_test_bureau,installment_amt_hist],axis=1)
df_test_bureau.drop(['INSTALLMENT_HIST','Installment_Frequency'],axis=1,inplace=True)


del installment_amt_hist,ddp_hist,amt_paid_hist,amt_overdue_hist,cur_bal_hist

In [319]:
df_test_bureau.drop(['AMT PAID - HIST','AMT OVERDUE - HIST','CUR BAL - HIST','DPD - HIST'],axis=1,inplace=True)

In [320]:
df_train_bureau['Source'] = "Train"
df_test_bureau['Source'] = "Test"
df_bureau_data = pd.concat([df_train_bureau, df_test_bureau],axis=0)
df_bureau_data.shape

(624863, 32)

In [321]:
df_bureau_data.head()

,ID,SELF-INDICATOR,ACCT-TYPE,CONTRIBUTOR-TYPE,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,WRITE-OFF-AMT,Count_No_Delays_In_Payment,Count_Payment_Past_Due_Date,History_Not_Available_Curr_Month,History_Not_Available_Prior_Month,Min_Cur_Bal,Max_Cur_Bal,Sum_Cur_Bal,Avg_Cur_Bal,Sum_Successive_Diff_CurBal,Avg_Successive_Diff_CurBal,Min_Overdue,Max_Overdue,Sum_Overdue,Avg_Overdue,Sum_Successive_Diff_Overdue,Avg_Successive_Diff_Overdue,Min_Amount_Paid,Max_Amount_Paid,Sum_Amount_Paid,Avg_Amount_Paid,Installment_Amount,Source
0,1.0,False,Overdraft,NAB,Individual,Delinquent,37352.0,,37873.0,0.0,1.0,1.0,0.0,0.0,0.0,37873.0,50693.0,16897.666667,-37873.0,-18936.500000,0.0,37873.0,37873.0,12624.333333,-37873.0,-18936.5,0,0,0,0.0,,Train
1,1.0,False,Auto Loan (Personal),NAB,Individual,Active,44000.0,"1,405/Monthly",20797.0,0.0,21.0,1.0,0.0,0.0,0.0,44601.0,721702.0,31378.347826,-20797.0,-945.318182,0.0,1452.0,1452.0,63.130435,0.0,0.0,0,0,0,0.0,"1,405",Train
2,1.0,True,Tractor Loan,NBF,Individual,Active,145000.0,,116087.0,0.0,6.0,0.0,0.0,0.0,0.0,145000.0,812174.0,116024.857143,-116087.0,-19347.833333,0.0,0.0,0.0,0.000000,0.0,0.0,0,0,0,0.0,,Train
3,1.0,True,Auto Loan (Personal),NBF,Individual,Closed,300000.0,,0.0,0.0,14.0,21.0,1.0,0.0,0.0,232582.0,4398411.0,118875.972973,0.0,0.000000,0.0,1014.0,16989.0,471.916667,0.0,0.0,0,0,0,0.0,,Train
4,1.0,True,Tractor Loan,NBF,Individual,Closed,275000.0,,0.0,0.0,36.0,0.0,0.0,0.0,0.0,226605.0,4254371.0,114983.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,0,0,0.0,,Train


**EDA - Main Data**

In [322]:
df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month,Source
0,1,Monthly,Arrear,Closed,PDC_E,1,NaN,48,450000,275000.0,275000.0,24000.0,2012-02-10,2016-01-15,2012-02-10,4022465,1568.0,21946,61.11,M,49.0,35833.33,RAISEN,MADHYA PRADESH,464993.0,> 48 Months,Train
1,2,Monthly,Advance,Closed,PDC,333,BHOPAL,47,485000,350000.0,350000.0,10500.0,2012-03-31,2016-02-15,2012-03-31,4681175,1062.0,34802,70.00,M,23.0,666.67,SEHORE,MADHYA PRADESH,466001.0,No Top-up Service,Train
2,3,Quatrly,Arrear,Active,Direct Debit,1,NaN,68,690000,519728.0,519728.0,38300.0,2017-06-17,2023-02-10,2017-06-17,25328146,1060.0,127335,69.77,M,39.0,45257.00,BHOPAL,MADHYA PRADESH,462030.0,12-18 Months,Train
3,7,Monthly,Advance,Closed,Billed,125,GUNA,48,480000,400000.0,400000.0,11600.0,2013-11-29,2017-11-10,2013-11-29,13021591,1060.0,25094,80.92,M,24.0,20833.33,ASHOK NAGAR,MADHYA PRADESH,473335.0,> 48 Months,Train
4,8,Monthly,Arrear,Closed,Billed,152,BILASPUR,44,619265,440000.0,440000.0,15000.0,2011-12-08,2015-07-05,2011-12-08,3291320,1046.0,21853,71.05,M,56.0,27313.67,BILASPUR,CHATTISGARH,495442.0,36-48 Months,Train


In [323]:
for col in df.columns:
    if df[col].dtype == "object":
        print ("Value Counts of {} column are:\n{}".format(col,df[col].value_counts(dropna=False)))

Value Counts of Frequency column are:
Half Yearly    87101
Monthly        33534
Quatrly        22303
BI-Monthly       462
Name: Frequency, dtype: int64
Value Counts of InstlmentMode column are:
Arrear     136586
Advance      6814
Name: InstlmentMode, dtype: int64
Value Counts of LoanStatus column are:
Closed    105723
Active     37677
Name: LoanStatus, dtype: int64
Value Counts of PaymentMode column are:
ECS             35143
Direct Debit    35073
PDC             30148
Billed          29311
PDC_E           10927
Auto Debit        951
SI Reject         843
Cheque            509
ECS Reject        480
PDC Reject          8
Escrow              7
Name: PaymentMode, dtype: int64
Value Counts of Area column are:
NaN                    12199
LUCKNOW                 9547
SIRSA                   7085
NELLORE                 6259
INDORE                  5674
                       ...  
UTTAR DINAJPUR             2
KOLKATA WOOD STREET        1
KHAMGAON                   1
JALORE                  

In [324]:
# We can remove the following columns, same way as we did earlier
df.drop(['Area','City','BranchID','AssetID','ManufacturerID','SupplierID','City','ZiPCODE'], axis=1, inplace=True)

# Replacing some values in some columns based on their occurences, same as we did earlier
df['PaymentMode'].replace({"SI Reject":"Reject",
                           "ECS Reject":"Reject",
                           "PDC Reject":"Reject",
                           "Cheque":"PDC",
                           "PDC_E":"PDC",
                           "Escrow":"Reject",
                           "Auto Debit":"Direct Debit"}, inplace=True)

df['State'].replace({"HIMACHAL PRADESH":"Others",
                     "JHARKHAND":"Others",
                     "ASSAM":"Others",
                     "DELHI":"Others",
                     "CHANDIGARH":"Others",
                     "TAMIL NADU":"Others",
                     "DADRA AND NAGAR HAVELI":"Others"}, inplace=True)

df['Top-up Month'].replace({"12-18 Months":0,
                            "18-24 Months":1,
                            "24-30 Months":2,
                            "30-36 Months":3,
                            "36-48 Months":4,
                            " > 48 Months":5,
                            "No Top-up Service":6}, inplace=True)

In [325]:
df.isna().sum()

ID                     0
Frequency              0
InstlmentMode          0
LoanStatus             0
PaymentMode            0
Tenure                 0
AssetCost              0
AmountFinance          0
DisbursalAmount        0
EMI                    0
DisbursalDate          0
MaturityDAte           1
AuthDate               0
LTV                    0
SEX                   64
AGE                   66
MonthlyIncome        248
State                  0
Top-up Month       14745
Source                 0
dtype: int64

In [326]:
df.drop(df[df['MaturityDAte'].isna()].index,axis=0,inplace=True)
df['SEX'].fillna("Missing",inplace=True)

#Replace missing 'AGE' values and 'MonthlyIncome' values with mean
df['AGE'].fillna(df['AGE'].mean(),inplace=True)
df['MonthlyIncome'].fillna(df['MonthlyIncome'].mean(),inplace=True)

In [327]:
df_bureau_data['Installment_Amount'].fillna(0,inplace=True)
df_bureau_data['Installment_Amount'] = df_bureau_data['Installment_Amount'].str.replace(",","")
df_bureau_data['Installment_Amount'] = pd.to_numeric(df_bureau_data['Installment_Amount'])

**Aggregate function based additional features - make a common function out of these for modular code**

In [328]:
def agg_func(df, df_bureau_data, col):
  agg_func_dict = {
      col : ['min', 'max', 'mean', 'sum']
  }

  agg_func_data = df_bureau_data.groupby('ID').agg(agg_func_dict)
  agg_func_data.columns = ['ID_' + ('_'.join(col).strip()) for col in agg_func_data.columns]
  agg_func_data.reset_index(inplace=True)
  df = df.merge(agg_func_data, on=['ID'], how='left')

  return df

In [329]:
df = agg_func(df, df_bureau_data, 'Count_No_Delays_In_Payment')

In [330]:
df = agg_func(df, df_bureau_data, 'Count_Payment_Past_Due_Date')

In [331]:
df = agg_func(df, df_bureau_data, 'History_Not_Available_Curr_Month')

In [332]:
df = agg_func(df, df_bureau_data, 'History_Not_Available_Prior_Month')

In [333]:
df = agg_func(df, df_bureau_data, 'Min_Cur_Bal')

In [334]:
df = agg_func(df, df_bureau_data, 'Max_Cur_Bal')

In [335]:
df = agg_func(df, df_bureau_data, 'Sum_Cur_Bal')

In [336]:
df = agg_func(df, df_bureau_data, 'Avg_Cur_Bal')

In [337]:
df = agg_func(df, df_bureau_data, 'Sum_Successive_Diff_CurBal')

In [338]:
df = agg_func(df, df_bureau_data, 'Avg_Successive_Diff_CurBal')

In [339]:
df = agg_func(df, df_bureau_data, 'Min_Amount_Paid')

In [340]:
df = agg_func(df, df_bureau_data, 'Max_Amount_Paid')

In [341]:
df = agg_func(df, df_bureau_data, 'Sum_Amount_Paid')

In [342]:
df = agg_func(df, df_bureau_data, 'Avg_Amount_Paid')

In [343]:
df = agg_func(df, df_bureau_data, 'Installment_Amount')

In [344]:
df = agg_func(df, df_bureau_data, 'DISBURSED-AMT/HIGH CREDIT')

In [345]:
df = agg_func(df, df_bureau_data, 'CURRENT-BAL')

In [346]:
df['Top-up Month'].fillna(0, inplace=True)

**Creating Further Additional Features**

In [347]:
self_indicator_dict = df_bureau_data.groupby("ID")['SELF-INDICATOR'].sum().to_dict()
acc_type_dict = df_bureau_data.groupby("ID")['ACCT-TYPE'].nunique().to_dict()
cont_type_dict = df_bureau_data.groupby("ID")['CONTRIBUTOR-TYPE'].nunique().to_dict()

In [348]:
df['Count_Self_Indicator'] = df["ID"].map(self_indicator)
df['Account_Type_Unique'] = df["ID"].map(acc_type_dict)
df['Unique_Contributor_Unique'] = df["ID"].map(cont_type_dict)

In [349]:
# Creating some more features, leveraging data of one table and fitting into another.
df['Common_ACCT-TYPE_By_ID_mode'] = df['ID'].map(df_bureau_data.groupby("ID")['ACCT-TYPE'].apply(lambda x: x.mode()[0]))
df['Common_CONTRIBUTOR-TYPE_By_ID_mode'] = df['ID'].map(df_bureau_data.groupby("ID")['CONTRIBUTOR-TYPE'].apply(lambda x: x.mode()[0]))
df['Common_OWNERSHIP-IND_By_ID_mode'] = df['ID'].map(df_bureau_data.groupby("ID")['OWNERSHIP-IND'].apply(lambda x: x.mode()[0]))
df['Common_ACCOUNT-STATUS_By_ID_mode'] = df['ID'].map(df_bureau_data.groupby("ID")['ACCOUNT-STATUS'].apply(lambda x: x.mode()[0]))

**Categorical Value Encoding**

In [ ]:
for col in df.columns:
  if df[col].dtype == 'object':
    df[col] = np.where(df[col].isnull(), "Missing", df[col])

In [350]:
#Target-Oriented Mean Encoding
payment_mode_series = df.groupby('Frequency')['Top-up Month'].mean()

In [353]:
#Target-Oriented Mean Encoding
payment_mode_dict = payment_mode_series.to_dict()

In [354]:
#Target-Oriented Mean Encoding
df['Frequency_encode'] = df['Frequency'].map(payment_mode_dict)
df.drop(['Frequency'], axis=1, inplace=True)

In [ ]:
for col in df.columns:
  if df[col].dtype == 'object':
    df[col] = np.where(df[col].isnull(), "Missing", df[col])

In [375]:
df = pd.get_dummies(df, drop_first=True)

In [378]:
df.head()

,ID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,LTV,AGE,MonthlyIncome,ID_Count_No_Delays_In_Payment_min,ID_Count_No_Delays_In_Payment_max,ID_Count_No_Delays_In_Payment_mean,ID_Count_No_Delays_In_Payment_sum,ID_Count_Payment_Past_Due_Date_min,ID_Count_Payment_Past_Due_Date_max,ID_Count_Payment_Past_Due_Date_mean,ID_Count_Payment_Past_Due_Date_sum,ID_History_Not_Available_Curr_Month_min,ID_History_Not_Available_Curr_Month_max,ID_History_Not_Available_Curr_Month_mean,ID_History_Not_Available_Curr_Month_sum,ID_History_Not_Available_Prior_Month_min,ID_History_Not_Available_Prior_Month_max,ID_History_Not_Available_Prior_Month_mean,ID_History_Not_Available_Prior_Month_sum,ID_Min_Cur_Bal_min,ID_Min_Cur_Bal_max,ID_Min_Cur_Bal_mean,ID_Min_Cur_Bal_sum,ID_Max_Cur_Bal_min,ID_Max_Cur_Bal_max,ID_Max_Cur_Bal_mean,ID_Max_Cur_Bal_sum,ID_Sum_Cur_Bal_min,ID_Sum_Cur_Bal_max,ID_Sum_Cur_Bal_mean,ID_Sum_Cur_Bal_sum,ID_Avg_Cur_Bal_min,ID_Avg_Cur_Bal_max,ID_Avg_Cur_Bal_mean,ID_Avg_Cur_Bal_sum,ID_Sum_Successive_Diff_CurBal_min,ID_Sum_Successive_Diff_CurBal_max,ID_Sum_Successive_Diff_CurBal_mean,ID_Sum_Successive_Diff_CurBal_sum,ID_Avg_Successive_Diff_CurBal_min,ID_Avg_Successive_Diff_CurBal_max,ID_Avg_Successive_Diff_CurBal_mean,ID_Avg_Successive_Diff_CurBal_sum,ID_Min_Amount_Paid_min,ID_Min_Amount_Paid_max,ID_Min_Amount_Paid_mean,ID_Min_Amount_Paid_sum,ID_Max_Amount_Paid_min,ID_Max_Amount_Paid_max,ID_Max_Amount_Paid_mean,ID_Max_Amount_Paid_sum,ID_Sum_Amount_Paid_min,ID_Sum_Amount_Paid_max,ID_Sum_Amount_Paid_mean,ID_Sum_Amount_Paid_sum,ID_Avg_Amount_Paid_min,ID_Avg_Amount_Paid_max,ID_Avg_Amount_Paid_mean,ID_Avg_Amount_Paid_sum,ID_Installment_Amount_min,ID_Installment_Amount_max,ID_Installment_Amount_mean,ID_Installment_Amount_sum,ID_DISBURSED-AMT/HIGH CREDIT_min,ID_DISBURSED-AMT/HIGH CREDIT_max,ID_DISBURSED-AMT/HIGH CREDIT_mean,ID_DISBURSED-AMT/HIGH CREDIT_sum,ID_CURRENT-BAL_min,ID_CURRENT-BAL_max,ID_CURRENT-BAL_mean,ID_CURRENT-BAL_sum,Account_Type_Unique,Unique_Contributor_Unique,Frequency_encode,InstlmentMode_Arrear,LoanStatus_Closed,PaymentMode_Direct Debit,PaymentMode_ECS,PaymentMode_PDC,PaymentMode_Reject,SEX_M,SEX_Missing,State_BIHAR,State_CHATTISGARH,State_GUJARAT,State_HARYANA,State_KARNATAKA,State_MADHYA PRADESH,State_MAHARASHTRA,State_ORISSA,State_Others,State_PUNJAB,State_RAJASTHAN,State_TELANGANA,State_UTTAR PRADESH,State_UTTARAKHAND,State_WEST BENGAL,Source_Train,Count_Self_Indicator_1,Count_Self_Indicator_2,Count_Self_Indicator_3,Count_Self_Indicator_4,Count_Self_Indicator_5,Count_Self_Indicator_6,Count_Self_Indicator_7,Count_Self_Indicator_8,Count_Self_Indicator_9,Count_Self_Indicator_10,Count_Self_Indicator_12,Count_Self_Indicator_17,Common_ACCT-TYPE_By_ID_mode_Business Loan - Secured,Common_ACCT-TYPE_By_ID_mode_Business Loan Against Bank Deposits,Common_ACCT-TYPE_By_ID_mode_Business Loan General,Common_ACCT-TYPE_By_ID_mode_Business Loan Priority Sector Agriculture,Common_ACCT-TYPE_By_ID_mode_Business Loan Priority Sector Others,Common_ACCT-TYPE_By_ID_mode_Business Loan Priority Sector Small Business,Common_ACCT-TYPE_By_ID_mode_Business Loan Unsecured,Common_ACCT-TYPE_By_ID_mode_Business Non-Funded Credit Facility-Priority Sector- Small Business,Common_ACCT-TYPE_By_ID_mode_Business Non-Funded Credit Facility-Priority Sector-Agriculture,Common_ACCT-TYPE_By_ID_mode_Commercial Vehicle Loan,Common_ACCT-TYPE_By_ID_mode_Construction Equipment Loan,Common_ACCT-TYPE_By_ID_mode_Consumer Loan,Common_ACCT-TYPE_By_ID_mode_Credit Card,Common_ACCT-TYPE_By_ID_mode_Education Loan,Common_ACCT-TYPE_By_ID_mode_Gold Loan,Common_ACCT-TYPE_By_ID_mode_Housing Loan,Common_ACCT-TYPE_By_ID_mode_Individual,Common_ACCT-TYPE_By_ID_mode_JLG Individual,Common_ACCT-TYPE_By_ID_mode_Kisan Credit Card,Common_ACCT-TYPE_By_ID_mode_Loan Against Bank Deposits,Common_ACCT-TYPE_By_ID_mode_Loan Against Shares / Securities,Common_ACCT-TYPE_By_ID_mode_Loan to Professional,Common_ACCT-TYPE_By_ID_mode_Microfinance Business Loan,Common_ACCT-

**Date-Time Value Encoding/Cleaning:**

In [380]:
df['DisbursalDate_Year'] = df['DisbursalDate'].dt.year
df['MaturityDAte_Quarter'] = df['MaturityDAte'].dt.quarter 
df['DisbursalMonth_Month'] = df['DisbursalDate'].dt.month  
df['DisbursalDate_Month_Start']=df['DisbursalDate'].dt.is_month_start
df['DisbursalDate_Month_End']=df['DisbursalDate'].dt.is_month_end

df['MaturityYear']=df['MaturityDAte'].dt.year
df['MaturityQuarter']=df['MaturityDAte'].dt.quarter
df['MaturityMonth']=df['MaturityDAte'].dt.month
df['MaturityDate_is_Month_Start']=df['MaturityDAte'].dt.is_month_start
df['MaturityDate_is_Month_End']=df['MaturityDAte'].dt.is_month_end

In [382]:
df.drop(['DisbursalDate', 'MaturityDAte'], axis=1, inplace=True)

In [384]:
df = pd.get_dummies(df, drop_first=True)

**Yes You Can**
**Advance Feature Enginnering - only later**

In [392]:
var_threshold = VarianceThreshold(threshold=0.1)

In [394]:
var_threshold
df.columns[var_threshold.get_support()]
len(df.columns)

TypeError: ignored